In [1]:
import pandas as pd
import numpy as np
from plotnine import *

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import GaussianNB, CategoricalNB

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelBinarizer #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv

from sklearn.metrics import accuracy_score, confusion_matrix,\
 f1_score, recall_score, precision_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

# Together

## Naive Bayes

Let's use the Naive Bayes algorithm to predict the mode of Beyonce songs, using the predictors energy, danceability, and valence.





In [3]:
beyonce = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/Beyonce_data.csv")
beyonce.head()

TODO: What are the data types of the predictors? What kind of NB should we use?

In [6]:
predictors = ["energy", "danceability", "valence"]

X = beyonce[predictors]
y = beyonce["mode"]

# TTS
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# Z-score
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# create model
nb = # TODO

nb.fit(X_train, y_train)

y_pred_train = nb.predict(X_train)
y_pred_test = nb.predict(X_test)

y_pred_train_prob = nb.predict_proba(X_train)[:,1]
y_pred_test_prob = nb.predict_proba(X_test)[:,1]

In [7]:
# assess
print("Train Acc       : ", accuracy_score(y_train, y_pred_train))
print("Train Prescision: ", precision_score(y_train, y_pred_train))
print("Train Recall    : ", recall_score(y_train, y_pred_train))
print("Train F1        : ", f1_score(y_train, y_pred_train))
print("Train ROC AUC   : ", roc_auc_score(y_train, y_pred_train_prob))
print()
print("Test Acc        : ", accuracy_score(y_test, y_pred_test))
print("Test Prescision : ", precision_score(y_test, y_pred_test))
print("Test Recall     : ", recall_score(y_test, y_pred_test))
print("Test F1         : ", f1_score(y_test, y_pred_test))
print("Test ROC AUC    : ", roc_auc_score(y_test, y_pred_test_prob))

## KNN

Let's do the same prediction, excpet using a KNN algorithm.

Remember, we'll need to select the hyperparameter n_neighbors, and we'll do this using a Grid Search.

TODO: Discussion - why do we need to use a validation set?

In [8]:
# load in + prepare data
beyonce = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/Beyonce_data.csv")

predictors = ["energy", "danceability", "valence"]
X = beyonce[predictors]
y = beyonce["mode"]

# TTS
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# Z-score
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# define KNN Model

knn = # TODO

# Select values you want to search over
hparams = # TODO

# define your GridSearchCV()
grid = GridSearchCV(#, #, scoring = "precision", cv = 5, refit = True)

grid.fit(X_train, y_train)

print("GridSearchCV chose: ", grid.best_estimator_.get_params()["n_neighbors"])


In [ ]:
# predict
y_pred_train = grid.predict(X_train)
y_pred_test = grid.predict(X_test)

y_pred_train_prob = grid.predict_proba(X_train)[:,1]
y_pred_test_prob = grid.predict_proba(X_test)[:,1]

In [9]:
# assess
print("Train Acc       : ", accuracy_score(y_train, y_pred_train))
print("Train Prescision: ", precision_score(y_train, y_pred_train))
print("Train Recall    : ", recall_score(y_train, y_pred_train))
print("Train F1        : ", f1_score(y_train, y_pred_train))
print("Train ROC AUC   : ", roc_auc_score(y_train, y_pred_train_prob))
print()
print("Test Acc        : ", accuracy_score(y_test, y_pred_test))
print("Test Prescision : ", precision_score(y_test, y_pred_test))
print("Test Recall     : ", recall_score(y_test, y_pred_test))
print("Test F1         : ", f1_score(y_test, y_pred_test))
print("Test ROC AUC    : ", roc_auc_score(y_test, y_pred_test_prob))

TODO: Discussion - which model (KNN or NB) performed better for this dataset? How did you make that decision?

### Decision Boundaries Plots


We can use sklearn's built in DecisionBoundaryDisplay() to view the boundaries we've made. Note: We can only do this in 2D, so we'll select two of our predictors to look at

In [10]:
X = X[['energy', 'danceability']]

# note - the amount you add/subtract (0.05) should be dependent on the scale of your column
# our columns have very small numbers, so I set it to be very small
feature_1, feature_2 = np.meshgrid(
    np.linspace(X['energy'].min() - 0.05, X['energy'].max() + 0.05),
    np.linspace(X['danceability'].min() - 0.05, X['danceability'].max() + 0.05)
)

grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T

knn = KNeighborsClassifier(n_neighbors = 3).fit(X, y)

y_pred = np.reshape(knn.predict(grid), feature_1.shape)
display = DecisionBoundaryDisplay(
    xx0=feature_1, xx1=feature_2, response=y_pred
)
display.plot()

display.ax_.scatter(
    X['energy'], X['danceability'], c=y, edgecolor="black"
)

display.ax_.set_title("KNN Decision Boundary (k = 3)")
display.ax_.set_xlabel('Energy')
display.ax_.set_ylabel('danceability')
display.ax_.legend()

# ICA

## Wine Data

Download the [wine quality dataset](https://www.kaggle.com/datasets/ruthgn/wine-quality-data-set-red-white-wine). The code below creates a new column, is_white, which has a 1 where a wine is white and a 0 otherwise. Build both a KNN and a NB model to predict the is_white column.



In [11]:
data = pd.read_csv(# TODO - link)

data['is_white'] = (data['type'] == 'white').astype(int)
data.head()

TODO: Which kind of NB will you use here? Why?

In [ ]:
# NB



TODO: Discuss the performance of your NB Model

In [ ]:
# KNN



TODO: Which n_neighbors was best? Discuss the performance of your KNN model. Which performed better, KNN or NB?

## Witch Data

We are going to use some data about alleged witches to predict whether someone is a witch. Load in the salem.csv file from GitHub. The dataset contains the following variables:
* age
* marital_status: Single, Widowed, Married, Divorced
* occupation: Farmer, Midwife, Homemaker, Merchant
* pet_ownership: binary indicator of if person owns pet
* unusual_knowledge: binary indicator of if person has unusual knowlege
* num_children
* acres_owned: # of acres owned by person
* income: income in dollars
* years_of_education
* is_witch: binary variable indicating if subject is a witch

In [12]:
witches = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/salem.csv")
witches.head()

### NB

TODO: Use GaussianNB to predict the is_witch column using all of the continuous predictors

In [ ]:
# GaussianNB


TODO: Use CategoricalNB to predict the is_witch column using marital_status and occupation. Hint: You'll need to use dummy variables/one hot encoding.

In [ ]:
# CategoricalNB


### KNN
TODO: Build a KNN model to predict is_witch using all predictors. What was the K chosen by grid search?

In [ ]:
# KNN

TODO: Which of those 3 models performed the best? Why?

### Trees

We discussed how decision trees have more than one hyperparameter to check (min_samples_leaf and max_depth). Let's get practice using GridSearchCV for more than one hyperparameter.


Use all of the predictors to build a DecisionTree.

Try the hyperparameter options of:
max_depth : [2, 3, 4, 5]
min_samples_leaf: [2, 3, 4]

In [13]:
predictors = ["age", "marital_status", "occupation", "pet_ownership",	"unusual_knowledge", "num_children", "acres_owned",	"income",	"years_of_education"]
contin = ["age", "num_children", "acres_owned", "income", "years_of_education"]
categ = ["marital_status", "occupation"]

X = witches[predictors]
y = witches["is_witch"]

# dummy categorical predictors
X = pd.get_dummies(X, columns=categ)

# TTS
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# Z-score continuous
# Z-score
scaler = StandardScaler()
X_train[contin] = scaler.fit_transform(X_train[contin])
X_test[contin] = scaler.transform(X_test[contin])


# Define Decision Tree

tree = DecisionTreeClassifier()

# Select values you want to search over
hparams = {"max_depth": # TODO: fill in desired values,
           "min_samples_leaf": # TODO: fill in desired values}

# define your GridSearchCV()
grid = GridSearchCV(tree, hparams, scoring = "precision", cv = 5, refit = True)


grid.fit(X_train, y_train)

print("GridSearchCV chose:", grid.best_estimator_.get_params()["max_depth"], "max depth")
print("GridSearchCV chose:", grid.best_estimator_.get_params()["min_samples_leaf"], "min samples leaf")

TODO: Which hyperparameters were chosen?